<a href="https://colab.research.google.com/github/adityamavle/Domain_Specific_Financial_LLM/blob/master/llm_semantic_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [ ]:
import os
import re
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
device_map = {"": 0}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [ ]:
def generate_text(model_name, tokenizer, prompt):
    logging.set_verbosity(logging.CRITICAL)
    # Initialize the pipeline with the provided model and tokenizer
    pipe = pipeline(task="text-generation", model=model_name, tokenizer=tokenizer, max_length=1024)
    # Run the pipeline with the formatted prompt
    #print("The prompt is",prompt)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    parts = result[0]['generated_text'].split("[/INST] ")
    #answer = parts[1].split(".")[0]
    #print("The Response is:",parts[0][1])
    # Print the prompt and the generated text
    return parts[1]

In [ ]:
reference_answer = "The potential effects of adverse macroeconomic conditions on the Company's operations and performance include: 1. Adversely impacting consumer confidence and spending, leading to a decrease in demand for the Company's products and services. 2. Financial instability and difficulties in obtaining credit for the Company's suppliers, contract manufacturers, logistics providers, distributors, cellular network carriers, and other channel partners. 3. Increased credit and collectibility risk on the Company's trade receivables. 4. The failure of derivative counterparties and other financial institutions. 5. Limitations on the Company's ability to issue new debt. 6. Reduced liquidity. 7. Declines in the fair value of the Company's financial instruments. These factors can materially and adversely affect the Company's business, results of operations, financial condition, and stock price."

generated_answer = "<INST> What are the potential effects of adverse macroeconomic conditions on the Company's operations and performance?</INST> <INPUT>. As a result, the Company’s operations and performance depend significantly on global and regional economic conditions. Adverse macroeconomic conditions, including inflation, slower growth or recession, new or increased tariffs and other barriers to trade, changes to fiscal and monetary policy, tighter credit, higher interest rates, high unemployment and currency fluctuations can adversely impact consumer confidence and spending and materially adversely affect demand for the Company’s products and services. In addition, consumer confidence and spending can be materially adversely affected in response to financial market volatility, negative financial news, conditions in the real estate and mortgage markets, declines in income or asset values, energy shortages and cost increases, labor and healthcare costs and other economic factors. In addition to an adverse impact on demand for the Company’s products, uncertainty about, or a decline in, global or regional economic conditions can have a significant impact on the Company’s suppliers, including: * Disruptions in the supply chain, including delays or increased costs in the delivery of raw materials, components or finished goods * Increased costs or reduced access to raw materials, components or finished goods * Reduced demand for the Company’s products and services * Increased costs or reduced access to financing * Reduced consumer spending and demand for the Company’s products and services * Increased competition and pricing pressures * Reduced profitability and cash flow * Increased risk of default or bankruptcy * Reduced access to markets and customers * Increased risk of asset impairment and write-downs * Reduced ability to invest in research and development and other growth initiatives * Reduced ability to respond to changing market conditions and customer preferences * Reduced ability to attract and retain key personnel * Reduced ability to maintain relationships with suppliers, customers and other stakeholders * Reduced ability to maintain compliance with laws and regulations * Reduced ability to maintain the Company’s reputation and brand image * Reduced ability to maintain the Company’s financial and operational controls * Reduced ability to maintain the Company’s risk management and compliance programs * Reduced ability to maintain the Company’s strategic partnerships and alliances * Reduced ability to maintain the Company’s intellectual property and other assets * Reduced ability to maintain the Company’s operations and facilities * Reduced ability to maintain the Company’s financial and operational flexibility * Reduced ability to respond to changes in the market and customer preferences * Reduced ability to respond to changes in the competitive landscape * Reduced ability to respond to changes in the regulatory environment * Reduced ability to respond to changes in the economic environment * Reduced ability to respond to changes in the political environment * Reduced ability to respond to changes in the social and cultural environment * Reduced ability to respond to changes in the technological environment * Reduced ability to respond to changes in the environmental and sustainability environment * Reduced ability to respond to changes in the security and privacy environment"

prompt =f"Given a financial dataset for evaluating LLMs, you will be provided with two paragraphs: one as the reference answer and the other as the generated answer by the LLM. \
      These answers are generated after providing a query and context to the LLM. Your task is to assess the relevance of the generated answer.Relevance here refers to the selection of important content from the source, ensuring that the summary includes only important information and avoids redundancies or excess information.\
      Reference Answer: {reference_answer} \
      Generated Answer: {generated_answer} \
      Evaluate the relevance of the generated answer and assign a score from 1 to 5 based on how well it covers the main points of the reference answer, considering whether it contains irrelevant or redundant information."

x = generate_text(base_model, tokenizer, prompt)

In [ ]:
if x:
  print("true")
else:
  print("false")

false


In [ ]:
import re

In [ ]:
import pandas as pd
df = pd.read_csv("/content/eval_data.csv")

In [ ]:
df = df.head(10)

In [ ]:
def eval_fluency(base_model, df):
  fluency_values = []
  for index, row in df.iterrows():
      reference_answer = row['Reference Answer']
      generated_answer = row['Predictions']
      prompt = f"Based on a financial dataset for evaluating the performance of LLMs in a financial domain-specific context,\
          you will be provided with two paragraphs: one as the reference answer and the other as the generated answer by the LLM. These answers are generated after providing a query and context to the LLM.  \
          Your task is to assess the fluency of the generated answer.\
          Fluency here refers to the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.\
          Generated Answer: {generated_answer} \
          Assign a score from 1 to 3 to evaluate the fluency of the text, based on the quality of the text, considering whether it contains errors that affect clarity or smoothness, and how easy it is to read and follow.\
          "
      ans = generate_text(base_model, tokenizer, prompt)
      if ans:
        score = re.search(r'\d', ans).group(0)
      else:
        score = "NAN"
      fluency_values.append(score)
      print(f"Row 1 done, score: {score}")
  df["Fluency"] = fluency_values

In [ ]:
def eval_consistency(base_model, df):
  consistency_values = []
  for index, row in df.iterrows():
      reference_answer = row['Reference Answer']
      generated_answer = row['Predictions']
      prompt = f"Given a financial dataset for evaluating LLMs, you will be provided with two paragraphs: one as the reference answer and the other as the generated answer by the LLM. \
      These answers are generated after providing a query and context to the LLM. Your task is to assess the consistency of the generated answer with respect to the reference answer. \
      Consistency here refers to the factual alignment between the two answers, ensuring that the generated answer contains only statements entailed by the reference answer and penalizing for any hallucinated facts.\
      Reference Answer: {reference_answer} \
      Generated Answer: {generated_answer} \
      Evaluate the consistency of the generated answer with respect to the reference answer and assign a score from 1 to 5 based on the extent of factual alignment between the two."
      ans = generate_text(base_model, tokenizer, prompt)
      if ans:
        score = re.search(r'\d', ans).group(0)
      else:
        score = "NAN"
      consistency_values.append(score)
  df["Consistency"] = consistency_values

In [ ]:
def eval_coherence(base_model, df):
  coherence_values = []
  for index, row in df.iterrows():
      reference_answer = row['Reference Answer']
      generated_answer = row['Predictions']
      prompt = f"Given a financial dataset for evaluating LLMs, you will be provided with two paragraphs: one as the reference answer and the other as the generated answer by the LLM. \
      These answers are generated after providing a query and context to the LLM. Your task is to assess the coherence of the generated answer. Coherence here refers to the collective quality of all sentences, focusing on whether the summary is well-structured, organized, and presents information in a logical order.\
      Reference Answer: {reference_answer} \
      Generated Answer: {generated_answer} \
      Evaluate the coherence of the generated answer and assign a score from 1 to 5 based on how well-structured and well-organized the summary is, ensuring it builds from sentence to a coherent body of information about the financial topic."
      ans = generate_text(base_model, tokenizer, prompt)
      if ans:
        score = re.search(r'\d', ans).group(0)
      else:
        score = "NAN"
      coherence_values.append(score)
  df["Coherence"] = coherence_values

In [ ]:
def eval_relevance(base_model, df):
  relevance_values = []
  for index, row in df.iterrows():
      reference_answer = row['Reference Answer']
      generated_answer = row['Predictions']
      prompt = f"Given a financial dataset for evaluating LLMs, you will be provided with two paragraphs: one as the reference answer and the other as the generated answer by the LLM. \
      These answers are generated after providing a query and context to the LLM. Your task is to assess the relevance of the generated answer.Relevance here refers to the selection of important content from the source, ensuring that the summary includes only important information and avoids redundancies or excess information.\
      Reference Answer: {reference_answer} \
      Generated Answer: {generated_answer} \
      Evaluate the relevance of the generated answer and assign a score from 1 to 5 based on how well it covers the main points of the reference answer, considering whether it contains irrelevant or redundant information."
      ans = generate_text(base_model, tokenizer, prompt)
      if ans:
        score = re.search(r'\d', ans).group(0)
      else:
        score = "NAN"
      relevance_values.append(score)
      print(score)
  df["Relevance"] = relevance_values

In [ ]:
eval_fluency(base_model,df)
eval_relevance(base_model,df)
eval_consistency(base_model,df)
eval_coherence(base_model,df)

4
4
4
4
3
NAN
4
4
4
NAN


In [ ]:
df.to_csv("eval_scores.csv")